# Spin Glass NADE

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dinesh110598/Spinglass_NADE/blob/main/main.ipynb)

In [1]:
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.math as tfm
import numpy as np
from IPython.display import clear_output
from numba import jit, prange

If you're running on your local machine, simply download the library.py and TrainData20k.npy files of the repo to the folder you're running this notebook on.

## Colab Instructions
Run the following cell if running on Google colaboratory notebook.

In [1]:
!curl -o library.py https://raw.githubusercontent.com/dinesh110598/Spinglass_NADE/main/library.py
!curl -o TrainingData20k.npy https://raw.githubusercontent.com/dinesh110598/Spinglass_NADE/main/TrainData20k.npy

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7774  100  7774    0     0   4970      0  0:00:01  0:00:01 --:--:--  4970
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    14  100    14    0     0      7      0  0:00:02  0:00:01  0:00:01     7


## Brief Introduction
We aim to construct a neural network (Neural Autoregressive Distribution Estimator) that can efficiently approximate the Boltzmann distribution of an EA spin glass system in equilibrium via the training data obtained using annealed MCMC simulations. See here for more details: https://arxiv.org/abs/2002.04292

The nitty gritty details of the neural network are coded in the library.py whose classes we'll be importing into this notebook directly:

In [2]:
from library import NADE_orig, NADE_fast

We have performed MCMC simulations of the EA lattice at T=0.5 externally and loading the numpy data for the same into here. Overall, we have 20000 latices of 20x20 EA lattices with Gaussian couplings to train our network with. In order to reduce the memory burden on the neural network, we made sure that *all latices have spins at (0,0) position = 1* and noting that multiplying the entire lattice by -1 gives an energetically equivalent configuration that the neural network need not learn

In [12]:
t_lattice = np.load ('Traindata20k.npy')
np.all (t_lattice[:,0,0] == 1) #Checks if (0,0) spins=1

True

Let's calculate the energy of each lattice in the training data after loading coupling constants from "NADEcouplings.npy" file:

In [9]:
Jnn = np.load ("NADE_couplings.npy")
Jnn.shape

(20, 20, 2)

In [10]:
@jit (nopython=True, parallel=True)
def calc_energy (Jnn, lattice):
    def bvc (x):
        if x == 20:
            return 0
        else:
            return x
    
    energy = np.zeros (lattice.shape[0], np.float32)
    for n in prange (lattice.shape[0]):
        for i in prange (lattice.shape[1]):
            for j in prange (lattice.shape[2]):
                energy[n] -= Jnn[i,j,0]*lattice[n,i,j]*lattice[n,i+1,j]
                energy[n] -= Jnn[i,j,1]*lattice[n,i,j]*lattice[n,i,j+1]
    return energy/400

Here, we convert the NumPy array to a tf.data datset so that we can batch and loop over it conveniently while training

In [7]:
train_data = tf.data.Dataset.from_tensor_slices (t_lattice)
train_data = train_data.batch(20)

### NADE_orig model
In the Reference 1, the neural network used there specified there evaluates the conditional probabilities of all N spins one by one. This network is built inside the NADE_orig class we've defined in *library*. Skip this section if it's too much waiting for training! Let's train and evaluate an object of this class: 

In [5]:
model = NADE_orig (inshape=(20,20),num_hidden=20)

We'll call train_step method of the NADE_orig class on a sample of the training data. This step builds the "autograph" associated with the method to provide significant speedup to subsequent executions of the method

In [10]:
loss = model.train_step (tf.constant (t_lattice[0:20,:,:]))
tf.print (loss)

TensorShape([])

Though we can use fit() method available for keras models, we'll quickly write a custom training loop to keep things transparent and more flexible:

In [12]:
epochs = 1
for epoch in range (epochs):
    for step, data in enumerate(train_data):
        loss = model.train_step (data)
        if step%50 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step+1, float(loss))
            )
            print("%d samples seen so far" % ((step + 1) * 20))

Training loss (for one batch) at step 1: 0.6892
20 samples seen so far
Training loss (for one batch) at step 21: 0.6900
420 samples seen so far
Training loss (for one batch) at step 41: 0.6902
820 samples seen so far
Training loss (for one batch) at step 61: 0.6899
1220 samples seen so far
Training loss (for one batch) at step 81: 0.6898
1620 samples seen so far


We can now sample from the distribution learnt by the model so far:

In [11]:
x = model.sample()

In [12]:
np.mean (x)

0.075

### NADE_fast model
The model originally prescribed in the paper is clearly too slow to train, particularly when we're looping over all N spins. This can however be speeded up by specifying a neural network that can evaluate all the N probabilities simultaneosly with the same architecture as originally prescribed in the paper. NADE_fast class here does the same:

In [8]:
fast_model = NADE_fast ((20,20), 20)

We can call the untrained model on a part of our training data. It returns values of probabilities around 0.5, with the way we've initialised the model.

In [9]:
fast_model.call (tf.constant(t_lattice[:20,:,:]))

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0.49699467, 0.50090426, 0.50145364, 0.50024694, 0.4977393 ,
       0.4973164 , 0.49811655, 0.49880856, 0.49776128, 0.49830493,
       0.5017334 , 0.49851722, 0.4992549 , 0.5011461 , 0.5000467 ,
       0.4996889 , 0.49932244, 0.49956924, 0.49838352, 0.49740195],
      dtype=float32)>

In [11]:
epochs = 10
for epoch in range (epochs):
    fast_model.loss_tracker.reset_states()
    for step, data in enumerate(train_data):
        loss = fast_model.train_step (data)
        if step%50 == 0:
            clear_output (wait=True)
            print("Epoch: %d" % (epoch,))
            print("Training loss (for one batch) at step %d: %.4f"
                % (step+1, float(loss)))
            print("%d samples seen so far" % ((step + 1) * 20))

Epoch: 9
Training loss (for one batch) at step 951: 0.3317
19020 samples seen so far


In [17]:
fast_model.save_weights ("FastNADEweights")

In [12]:
fast_model.call (t_lattice[60:80,:,:])

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0.8458787 , 0.8001719 , 0.72402114, 0.57857573, 0.8414492 ,
       0.7879002 , 0.75884783, 0.8207925 , 0.8414858 , 0.77756923,
       0.81860155, 0.50814086, 0.4804363 , 0.5107358 , 0.76328266,
       0.65623343, 0.7918186 , 0.8549846 , 0.5463747 , 0.47044188],
      dtype=float32)>

In [13]:
lattice = np.stack([fast_model.sample() for _ in range(20)])
lattice.shape

(20, 20, 20)

In [14]:
calc_energy (Jnn, lattice)

array([-1.1556046, -1.0963204, -1.065081 , -1.008458 , -1.0865479,
       -0.9758622, -1.0738789, -1.0997452, -1.0888323, -1.1030087,
       -1.0218664, -1.1054885, -1.0800701, -1.041604 , -1.0140287,
       -1.0544673, -1.0273954, -1.0291573, -1.0895194, -1.0061578],
      dtype=float32)

In [39]:
x = np.arange ()

0.02